# C-FIRE: Colombian Fire Incidence and Risk Estimator (Generación de la base pt. 2) 

### Feature engineering

#### Frecuencia de incendios por sector y fecha

In [ ]:
# CREACIÓN DE LA VARIABLE DE FECHAS (RANGO DEL 2020/01 AL 2024/09)
rango_fechas = pd.date_range(start='2020-01-01', end='2024-09-01', freq='MS')
rango_fechas

In [ ]:
# CREACIÓN DE VARIABLE DE SECTORES CON SECTORES ÚNICOS
sectores = mapa_final['sector'].unique()
sectores

In [ ]:
# UNIÓN DE SECTORES Y RANGO DE FECHAS
sectores_fechas = pd.MultiIndex.from_product([sectores, rango_fechas], names=['sector', 'mes']).to_frame(index=False)
sectores_fechas

In [ ]:
# CONTEO DE INCENDIOS POR SECTOR Y MES 
frecuencia_incendios = incendios_con_sector.groupby(['sector', 'fecha']).size().reset_index(name='frecuencia_incendios')
frecuencia_incendios

In [ ]:
# RENOMBRE DE VARIABLE 'mes' POR 'fecha'
sectores_fechas=sectores_fechas.rename(columns={'mes':'fecha'})
sectores_fechas

In [ ]:
# INCLUSIÓN DEL CONTEO DE INCENDIOS A LA BASE QUE SERÁ LA FINAL (DE MOMENTO CONTIENE SECTORES Y MESES)
basef_1 = pd.merge(sectores_fechas, frecuencia_incendios, on=['sector', 'fecha'], how='left')
basef_1

In [ ]:
# CREACIÓN DE LA VARIABLE BOOLEANA QUE MUESTRA SI HUBO INCENDIOS EN EL SECTOR Y EN EL MES ESPECÍFICO EN FUNCIÓN DE LOS REGISTROS VACIOS DEL CONTEO DE INCENDIOS
basef_1['si_hubo_incendio'] = np.where(basef_1['frecuencia_incendios'].isna(), 0, 1)
basef_1['frecuencia_incendios'] = basef_1['frecuencia_incendios'].fillna(0)
basef_1

In [ ]:
# COMPROBACIÓN DE QUE NO SEAN SOLO VALORES DE 0 EN LA VARIABLE BOOLEANA
print(basef_1['si_hubo_incendio'].max())

In [ ]:
# COMPROBACIÓN DE QUE NO SEAN SOLO VALORES DE 0 EN EL CONTEO DE INCENDIOS
print(basef_1['frecuencia_incendios'].max())

#### Distancia entre incendios por municipio y fecha

In [ ]:
# CONVERSIÓN Y RECTIFICACIÓN AL MISMO SISTEMA DE COORDENADAS
municipios=municipios.to_crs(epsg=4326)
print(incendios_final.crs, municipios.crs)

In [ ]:
# INCLUSIÓN DE INCENDIOS (GEOMETRÍA DE PUNTOS) A LOS MUNICIPIOS (GEOMETRÍA DE POLÍGONOS) 
incendios_con_municipios = gpd.sjoin(incendios_final, municipios[['municipio_cod', 'geometry']], how='left', predicate='within')
incendios_con_municipios

In [ ]:
# CONTEO DE REGISTROS VACIOS EN LA VARIABLE 'codigo' (INCENDIOS SIN MUNICIPIO)
nan_count = incendios_con_municipios['municipio_cod'].isna().sum()
nan_count

In [ ]:
# QUITAR REGISTROS VACIOS DE SECTOR Y FILTRACIÓN DE VARIABLES
incendios_con_municipios = incendios_con_municipios[incendios_con_municipios['municipio_cod'].notna()]
incendios_con_municipios = incendios_con_municipios.drop(columns=['brillo','index_right', 'latitud', 'longitud', 'acq_date', 'bright_t31', 'frp'])

incendios_con_municipios

In [ ]:
# CONVERSIÓN DE CRS DEL MAPA DE UNIÓN DE INCENDIOS POR MUNICIPIO AL MISMO CRS DEL MAPA DE SECTORES
incendios_con_municipios.to_crs(mapa_final.crs)

print(incendios_con_municipios.crs)

In [ ]:
# UNIÓN DE INCENDIOS POR MUNICIPIO CON EL MAPA DE SECTORES (UBICACIÓN DE PUNTOS DENTRO DE POLÍGONOS)
incendios_con_municipios_sector = gpd.sjoin(incendios_con_municipios, mapa_final[['sector', 'geometry']], how='left', predicate='within')
incendios_con_municipios_sector

In [ ]:
# CREAR LLAVE PARA UNIÓN POSTERIOR A LA BASE FINAL DE ACUERDO A LOS SECTORES (EL CAMBIO DE CRS ALTERA LAS COORDENADAS)
incendios_con_municipios_sector['llave_incendio'] = incendios_con_municipios_sector.index.astype(str)
incendios_con_municipios_sector

In [ ]:
# Asegurarnos de que los datos están transformados a coordenadas EPSG 3857
incendios_transformados = incendios_con_municipios_sector.to_crs(epsg=3857)

# Crear una lista para almacenar los resultados
resultados = []

# Agrupamos por municipio, año y mes
for (municipio_cod, año, mes), grupo_actual in incendios_transformados.groupby(['municipio_cod', 'año', 'mes']):
    # Determinar el mes anterior
    if mes == 1:  # Si es enero, retroceder al diciembre del año anterior
        año_anterior = año - 1
        mes_anterior = 12
    else:
        año_anterior = año
        mes_anterior = mes - 1

    # Filtrar los incendios del mismo municipio del mes anterior
    grupo_anterior = incendios_transformados[
        (incendios_transformados['municipio_cod'] == municipio_cod) &
        (incendios_transformados['año'] == año_anterior) &
        (incendios_transformados['mes'] == mes_anterior)
    ]
    
    # Si no hay incendios el mes anterior, llenar con valores predeterminados
    if grupo_anterior.empty:
        for llave in grupo_actual['llave_incendio']:
            resultados.append({
                'municipio_cod': municipio_cod,
                'año': año,
                'mes': mes,
                'llave_incendio': llave,
                'distancia_promedio': 999999,
                'distancia_minima': 999999,
                'distancia_maxima': 999999
            })
        continue

    # Coordenadas y llaves del grupo anterior y actual
    coords_anterior = grupo_anterior.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
    coords_actual = grupo_actual.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
    llaves_actual = grupo_actual['llave_incendio'].tolist()

    # Calcular la matriz de distancias entre incendios del grupo actual y del anterior
    dist_matrix = distance_matrix(coords_actual, coords_anterior)

    # Procesar cada incendio del grupo actual
    for i, llave in enumerate(llaves_actual):
        distancias = dist_matrix[i, :]
        
        if distancias.size > 0:
            promedio = distancias.mean()
            minimo = distancias.min()
            maximo = distancias.max()
        else:
            promedio = 999999
            minimo = 999999
            maximo = 999999

        resultados.append({
            'municipio_cod': municipio_cod,
            'año': año,
            'mes': mes,
            'llave_incendio': llave,
            'distancia_promedio': promedio,
            'distancia_minima': minimo,
            'distancia_maxima': maximo
        })

# Convertir resultados a DataFrame
distancia_por_incendio_mes_anterior = pd.DataFrame(resultados)

distancia_por_incendio_mes_anterior# CLASIFICACIÓN DE CÁLCULOS POR SECTOR
# Seleccionar solo las columnas de interés en distancia_por_incendio_mes_anterior
columnas_calculos = ['llave_incendio', 'distancia_promedio', 'distancia_minima', 'distancia_maxima']

# Realizar el merge manteniendo solo esas columnas adicionales
distancia_entre_incendios = pd.merge(
    incendios_con_municipios_sector, 
    distancia_por_incendio_mes_anterior[columnas_calculos], 
    on='llave_incendio', 
    how='outer'
)

distancia_entre_incendios

In [ ]:
# CÁLCULO DEL PROMEDIO POR SECTOR Y MES DE LA DISTANCIA PROMEDIO ENTRE INCENDIOS POR MUNICIPIO
# Agrupar por municipio_cod, año, mes y sector y calcular el promedio de las distancias promedio
distancia_entre_incendios = distancia_entre_incendios.groupby(['año', 'mes', 'sector'])['distancia_promedio'].mean().reset_index()

# Renombrar la columna para mayor claridad
distancia_entre_incendios.rename(columns={'distancia_promedio': 'promedio_distancia_promedio'}, inplace=True)

# Mostrar el DataFrame con los promedios de las distancias promedio por sector
distancia_entre_incendios

In [ ]:
# CREACIÓN DE VARIABLE DIA EN EL REGISTRO DE INCENDIOS CON SECTORES (PARA CREAR VARIABLE FECHA EN ESE GEODATAFRAME)
distancia_entre_incendios['dia']=1
distancia_entre_incendios

In [ ]:
# CREACIÓN DE VARIABLE 'fecha' Y DEFINICIÓN DE ESA VARIABLE COMO TAL 
distancia_entre_incendios['fecha'] = (
    distancia_entre_incendios['año'].astype(str) + '-' +
    distancia_entre_incendios['mes'].astype(str).str.zfill(2) + '-' +
    distancia_entre_incendios['dia'].astype(str).str.zfill(2)
)
distancia_entre_incendios['fecha'] = pd.to_datetime(distancia_entre_incendios['fecha'])
distancia_entre_incendios

In [ ]:
distancia_entre_incendios.promedio_distancia_promedio.min()

In [ ]:
# ADICIÓN DEL PROMEDIO DE LAS DISTANCIAS ENTRE INCENDIOS A LA BASE FINAL
distancia_entre_incendios = distancia_entre_incendios[['sector', 'fecha', 'promedio_distancia_promedio']].rename(
    columns={'promedio_distancia_promedio': 'distancia_promedio_entre_incendios'}
)

basef_1_1 = pd.merge(basef_1, distancia_entre_incendios, on=['sector', 'fecha'], how='left')

basef_1_1['distancia_promedio_entre_incendios'] = basef_1_1['distancia_promedio_entre_incendios'].fillna(999999)

basef_1_1

In [ ]:
basef_1_1.distancia_promedio_entre_incendios.min()

In [ ]:
basef_1_1.distancia_promedio_entre_incendios.mean()

In [ ]:
basef_1_1.distancia_promedio_entre_incendios.median()

In [ ]:
# COMPROBACIÓN DE VALORES EN LA MEDIA DE DISTANCIA ENTRE INCENDIOS
basef_1_1.distancia_promedio_entre_incendios.nunique()

#### Distancia vias e incendios

In [ ]:
# CONVERSIÓN A SISTEMA DE COORDENADAS SIMILAR DE LAS VIAS E INCENDIOS
vias = vias.to_crs(epsg=3116)
incendios_con_sector = incendios_con_sector.to_crs(epsg=3116)

print(incendios_con_sector.crs, vias.crs)

In [ ]:
# SIMPLIFICACIÓN DE LA GEOMETRÍA DE LAS LINEAS DE LAS VÍAS PARA ALIGERAR EL CÁLCULO Y REDUCIR EL COSTO COMPUTACIONAL
vias['geometry'] = vias['geometry'].simplify(tolerance=0.1, preserve_topology=True)

In [ ]:
# CREACIÓN DE FUNCIÓN PARA CALCULAR LA DISTANCIA ENTRE LOS INCENDIOS Y LAS VIAS
idx = index.Index()
# Agregar geometrías de vías al índice espacial
for i, geom in enumerate(vias.geometry):
    idx.insert(i, geom.bounds)  # Insertamos las geometrías en el índice con sus límites

# Función para calcular las vías cercanas con tolerancia ajustada
def encontrar_vias_cercanas(incendio_geom, vias, idx):
    # Obtener las vías cuyos límites intersectan con los límites del incendio
    possible_matches_index = list(idx.intersection(incendio_geom.bounds))
    
    # Filtrar las posibles coincidencias exactas
    possible_matches = vias.iloc[possible_matches_index]
    
    if possible_matches.empty:
        return 999999  # Si no hay coincidencias, devuelve 999999
    
    # Calcular la distancia mínima
    min_dist = possible_matches.geometry.distance(incendio_geom).min()
    return min_dist

# Aplicar la función para calcular distancias
incendios_con_sector['distancia_vias'] = incendios_con_sector.geometry.apply(lambda geom: encontrar_vias_cercanas(geom, vias, idx))

print(incendios_con_sector[['geometry', 'distancia_vias']].head())

In [ ]:
# COMPROBACIÓN DE QUE NO SOLO HAYA DISTANCIAS CON VALORES DE 9999 POR MEDIO DEL VALOR MÍNIMO 
incendios_con_sector.distancia_vias.min()

In [ ]:
# COMPROBACIÓN DE VALORES DE DISTANCIAS (USANDO LA VISUALIZACIÓN DE 'TODOS' LOS REGISTROS)
pd.set_option('display.max_rows', None)
print(incendios_con_sector['distancia_vias'])

In [ ]:
# DESHABILTACIÓN DE LA VISUALIZACIÓN DE 'TODOS' LOS REGISTROS
pd.reset_option('display.max_rows')

In [ ]:
# COMPROBACIÓN DE LA DISTANCIA ENTRE INCENDIOS Y VIAS POR MEDIO DE VALORES ÚNICOS
uv=incendios_con_sector['distancia_vias'].unique()
print(uv)

In [ ]:
# CONVERSIÓN DE INCENDIOS A UN SISTEMA DE COORDENADAS APROPIADO PARA UNIÓN CON LAS DISTANCIAS CON LAS VIAS
incendios_con_sector=incendios_con_sector.to_crs(epsg=4326)

mapa_final=mapa_final.to_crs(incendios_con_sector.crs)
print(mapa_final.crs, incendios_con_sector.crs)

In [ ]:
# CALCULO DEL PROMEDIO DE LA DISTANCIA DE INCENDIOS CON VIAS POR SECTOR Y MES
promedio_distancia = incendios_con_sector.groupby(['sector', 'fecha'])['distancia_vias'].mean().reset_index()
promedio_distancia.rename(columns={'distancia_vias': 'promedio_distancia_vias'}, inplace=True)
promedio_distancia

In [ ]:
# COMPROBACIÓN DE QUE NO SOLO HAYA DISTANCIAS CON VALORES DE 9999 POR MEDIO DEL VALOR MÍNIMO
promedio_distancia['promedio_distancia_vias'].min()

In [ ]:
# COMPROBACIÓN DE LA DISTANCIA ENTRE INCENDIOS Y VIAS POR MEDIO DE VALORES ÚNICOS
uv=promedio_distancia['promedio_distancia_vias'].unique()
print(uv)

In [ ]:
# UNIÓN DEL PROMEDIO DE LA DISTANCIA DE LOS INCENDIOS Y LAS VIAS A LA BASE FINAL POR MEDIO DE LAS VARIABLES 'sector' Y 'fecha'.
basef_2 = pd.merge(basef_1_1, promedio_distancia, on=['sector', 'fecha'], how='left')
basef_2['promedio_distancia_vias'] = basef_2['promedio_distancia_vias'].fillna(999999)
basef_2

In [ ]:
# COMPROBACIÓN DE QUE NO SOLO HAYA DISTANCIAS CON VALORES DE 9999 POR MEDIO DEL VALOR MÍNIMO
basef_2['promedio_distancia_vias'].min()

#### Conteo vías en sectores

In [ ]:
# CONTEO DE PRESENCIA DE VIAS POR SECTOR
vias_por_sector = sectores_vias.groupby('sector').size().reset_index(name='conteo_vias')
vias_por_sector

In [ ]:
# COMPROBACIÓN DEL CONTEO POR MEDIO DEL VALOR MÍNIMO Y MÁXIMO
print(vias_por_sector['conteo_vias'].max(), vias_por_sector['conteo_vias'].min())

In [ ]:
# UNIÓN DEL CONTEO DE VIAS POR SECTOR A LA BASE FINAL Y REORDENAMIENTO DE VARIABLES 
basef_3 = pd.merge(vias_por_sector, basef_2, on='sector', how='outer')
n_o = ['sector','fecha','conteo_vias','si_hubo_incendio','frecuencia_incendios','distancia_promedio_entre_incendios', 'promedio_distancia_vias']
basef_3 = basef_3[n_o]
basef_3

#### Moda del tipo de vía por sector

In [ ]:
# CALCULO DE MODA DEL TIPO DE VIA PRESENTE POR SECTOR
moda_vias = sectores_vias.groupby('sector')['tipo'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
).reset_index()
moda_vias

In [ ]:
moda_vias.tipo.unique()

In [ ]:
# UNIÓN DE LA MODA DEL TIPO DE VIA POR SECTOR A LA BASE FINAL
basef_4 = pd.merge(basef_3, moda_vias, on='sector', how='left')
basef_4 = basef_4.rename(columns={'tipo': 'moda_vias'})
basef_4

In [ ]:
# SE REVISA QUE NO HAYA DATOS VACIOS EN LA MODA DE VIAS POR SECTOR
nan_count = basef_4['conteo_vias'].isna().sum()
nan_count

In [ ]:
# SE REVISA QUE NO HAYA DATOS VACIOS EN EL CONTEO DE VIAS POR SECTOR
nan_count = basef_4['moda_vias'].isna().sum()
nan_count

In [ ]:
# SE RELLENA LA AUSENCIA DE VIAS (CONTEO) CON 0 PARA DATOS FALTANTES Y CON 36 (DE ACUERDO A LA CATEGORIZACIÓN ASIGNADA MÁS ARRIBA)PARA LA MODA
basef_4['conteo_vias'] = basef_4['conteo_vias'].fillna(0)
basef_4['moda_vias'] = basef_4['moda_vias'].fillna('None')
basef_4

#### Moda del uso de la tierra por sector

In [ ]:
# CÁLCULO DE LA MODA DEL USO DE TIERRA POR SECTORES PARA CLASIFICAR LOS SECTORES EN FUNCIÓN DEL USO
moda_uso_tierra = uso_tierra_sectores.groupby('sector')['Vocacion'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
).reset_index()
moda_uso_tierra

In [ ]:
# VISUALIZACIÓN DE DATOS FALTANTES LUEGO DE OBTENER LA MODA
nan_count = moda_uso_tierra['Vocacion'].isna().sum()
nan_count

In [ ]:
# UNIÓN DE LA MODA DEL USO DE LA TIERRA POR SECTOR 
basef_5 = pd.merge(basef_4, moda_uso_tierra, on='sector', how='left')
basef_5 = basef_5.rename(columns={'Vocacion': 'uso_tierra_moda'})
basef_5

In [ ]:
# VISUALIZACIÓN PARA COMPROBAR QUE SI SE HAYA CALCULADO LA MODA ADECUADAMENTE
basef_5.uso_tierra_moda.unique()

In [ ]:
# IMPUTACIÓN DE REGISTROS FALTANTES POR MEDIO DE LA MODA GENERAL
moda = basef_5['uso_tierra_moda'].mode()[0]  # Obtiene la moda (primera en caso de múltiples)
basef_5['uso_tierra_moda'] = basef_5['uso_tierra_moda'].fillna(moda)  # Llena los NA con la moda
basef_5.uso_tierra_moda.unique()

#### Conteo de puntos de minería ilegal por sector

In [ ]:
# CONTEO DE MINERIA POR SECTOR
conteo_mineria = mineria_sectores.groupby('sector').size().reset_index(name='conteo_mineria')
conteo_mineria

In [ ]:
# UNIÓN DEL CONTEO DE MINERIA POR SECTOR A LA BASE FINAL 
basef_6 = pd.merge(basef_5, conteo_mineria, on='sector', how='left')
#basef_6 = basef_6.rename(columns={'uso_tierra_numerico': 'uso_tierra_moda'})
basef_6

In [ ]:
# VISUALIZACIÓN DE VALORES FALTANTES
valores_nulos = basef_6.conteo_mineria.isna().sum()
print(valores_nulos)

In [ ]:
# IMPUTACIÓN DE VALORES DE 0 AL CONTEO DE MINERIA POR SECTOR
basef_6["conteo_mineria"] = basef_6["conteo_mineria"].fillna(0)
basef_6

In [ ]:
# COMPROBACIÓN DE VALORES ÚNICOS EN EL CONTEO DE MINERÍA POR SECTOR
basef_6.conteo_mineria.unique()

In [ ]:
# COMPROBACIÓN DE QUE HAYA CONTEO DE MIENRIA POR SECTOR Y NO SOLO VALORES DE 0 POR MEDIO DEL VALOR MÁXIMO
basef_6.conteo_mineria.max()

### Feature transformation

#### Conversión del tipo de vía a numérico

In [ ]:
# CAMBIO DE CATEGORIZACIÓN TEXTUAL DE VIAS POR NUMÉRICA
categoria_numerica = {
    'residencial': 1,         # Vías residenciales (áreas de viviendas)
    'primaria': 2,            # Vías primarias (importantes en la red vial)
    'terciaria': 3,           # Vías terciarias (conectan áreas locales)
    'secundaria': 4,          # Vías secundarias (menos tráfico que las primarias)
    'enlace_primaria': 5,     # Conexiones de vías primarias
    'enlace_principal': 6,    # Conexiones de vías principales
    'enlace_secundaria': 7,   # Conexiones de vías secundarias
    'principal': 8,           # Vía principal (vías rápidas, carreteras importantes)
    'sendero': 9,             # Caminos para peatones o bicicletas
    'servicio': 10,           # Vías de servicio (generalmente en áreas industriales)
    'enlace_terciaria': 11,   # Conexiones de vías terciarias
    'peatonal': 12,           # Zonas peatonales
    'no_clasificada': 13,     # No clasificada
    'camino': 14,             # Caminos sin pavimentar, usados para vehículos
    'escaleras': 15,          # Escaleras
    'ciclovía': 16,           # Ciclovías
    'camino_caballos': 17,    # Caminos para caballos
    'calle_residencial': 18,  # Calles residenciales o de baja velocidad
    'pista': 19,              # Pistas de carreras
    'en_construcción': 20,    # En construcción
    'plataforma': 21,         # Plataformas (paradas de buses, trenes)
    'rotonda': 22,            # Rotondas
    'autopista': 23,          # Autopista (máxima importancia)
    'parada_bus': 24,         # Paradas de buses
    'carretera': 25,          # Carreteras principales
    'propuesta': 26,          # Propuesta de vía (sin construcción todavía)
    'desconocido': 27,        # Desconocido o sin datos
    'área_descanso': 28,      # Áreas de descanso
    'servicios': 29,          # Áreas de servicios (como estaciones de servicio)
    'acceso_emergencia': 30,  # Acceso de emergencia
    'vado': 31,               # Cruces de ríos
    'enlace_autopista': 32,   # Conexiones de autopistas
    'guía_bus': 33,           # Carriles exclusivos para buses
    'minirotonda': 34,        # Mini rotondas
    'None': 35,
    None: 35                  # Valor faltante (nan)
}

# SE REEMPLAZA LA COLUMNA CON VARIABLE NUMÉRICAS
basef_6['moda_vias'] = basef_6['moda_vias'].map(categoria_numerica)

basef_6.head()

In [ ]:
# VISUALIZACIÓN DE VALORES FALTANTES
valores_nulos = basef_6.uso_tierra_moda.isna().sum()
print(valores_nulos)

In [ ]:
basef_6.moda_vias.unique()

#### Conversión de la moda del uso de la tierra a numérico

In [ ]:
# CATEGORIZACIÓN NUMÉRICA PARA EL USO DE LA TIERRA (SIN AGRUPAR)
uso_tierra_map = {
    'Agroforestal': 1,
    'Forestal': 2,
    'Zonas urbanas': 3,
    'Áreas Prioritarias para la Conservación': 4,
    'Agrícola': 5,
    'Ganadera': 6,
    'Áreas de Protección Legal': 7,
    'Conservación de Suelos': 8,
    'Cuerpo de agua': 9,
    'Aeropuerto': 10,
    'Arenal': 11,
    'Base militar': 12,
    'Basurero': 13,
    'Cantera': 14,
    'Edificación': 15,
    'Represa': 16,
    'Saladares': 17,
    'Fosa de mina de carbón': 18,
    'Tierra de relave de carbón': 19
}

# SE REEMPLAZA LA COLUMNA CON LAS CATEGORIAS NUMÉRICAS
basef_6['uso_tierra_moda'] = basef_6['uso_tierra_moda'].map(uso_tierra_map)

basef_6.head()

In [ ]:
basef_6['mes'] = basef_6['fecha'].dt.month
basef_6['año'] = basef_6['fecha'].dt.year

basef_6 = basef_6.drop(columns=['fecha'])
columnas = ['sector', 'año', 'mes'] + [col for col in basef_6.columns if col not in ['sector', 'año', 'mes']]
basef_6 = basef_6[columnas]

basef_6.head()

In [ ]:
# CONVERSIÓN DE VARIABLES NUMÉRICAS A CATEGÓRICAS
basef_6['mes'] = basef_6['mes'].astype('category')
basef_6['año'] = basef_6['año'].astype('category')
basef_6['sector'] = basef_6['sector'].astype('category')
basef_6['si_hubo_incendio'] = basef_6['si_hubo_incendio'].astype('category')
basef_6['moda_vias'] = basef_6['moda_vias'].astype('category')
basef_6['uso_tierra_moda'] = basef_6['uso_tierra_moda'].astype('category')

basef_6.head()

In [ ]:
# RECTIFICACIÓN DE LA CONVERSIÓN DE VARIABLES NUMÉRICAS A CATEGÓRICAS
basef_6.info()

In [ ]:
# GUARDADO DE BASE DE DATOS FINAL COMO CSV
basef_6.to_csv("Bases\Base_final_incendios.csv", index=False)

### Feature extraction

#### SelectKbest de Sklearn

In [ ]:
X = basef_6.drop(columns=['si_hubo_incendio']) 
y = basef_6['si_hubo_incendio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

selector = SelectKBest(score_func=f_classif, k=5)
X_new = selector.fit_transform(X_train, y_train)

print("Características seleccionadas:", X.columns[selector.get_support()])
print("Puntajes de las características:", selector.scores_)

La variable *promedio_distancia_vias* se elimina debido a su bajo puntaje de 56.63 en SelectKBest, lo que indica su baja relevancia para la predicción de incendios por sector y mes. Su eliminación ayuda a simplificar el modelo sin comprometer su precisión.

### Rebalanceo de clases

In [ ]:
under_sample = RandomUnderSampler (random_state=42)
x_resampled, y_resampled = under_sample.fit_resample(X_train, y_train)

In [ ]:
# Crear un conjunto de datos desequilibrado
X, y = make_classification(n_samples=500000, n_features=20, n_classes=2,
                           weights=[0.9, 0.1], random_state=42)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Mostrar la distribución de clases antes del rebalanceo
print("Distribución de clases antes del rebalanceo:")
print({label: count for label, count in zip(*np.unique(y_train, return_counts=True))})

# Definir la estrategia de rebalanceo: Sobremuestreo con SMOTE y Submuestreo de clases mayoritarias
over = SMOTE(sampling_strategy=0.5)  # Aumentar la clase minoritaria hasta el 50%
under = RandomUnderSampler(sampling_strategy=0.8)  # Reducir la clase mayoritaria al 80%
pipeline = Pipeline(steps=[('over', over), ('under', under)])

# Aplicar el rebalanceo
X_resampled, y_resampled = pipeline.fit_resample(X_train, y_train)

# Mostrar la distribución de clases después del rebalanceo
print("\nDistribución de clases después del rebalanceo:")
print({label: count for label, count in zip(*np.unique(y_resampled, return_counts=True))})

# Entrenar un modelo con los datos re-balanceados
model = RandomForestClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Evaluar el modelo
y_pred = model.predict(X_test)
print("\nReporte de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_pred))

# Universidad Externado de Colombia - Facultad de Economía
## Autores: Santiago A. Rodríguez Estrada & Laura S. Romero Suárez
## Revisado por: Daniel Godoy